# 1.事前準備

在我們開始寫神經網路之前，還沒安裝的話要先安裝gradio這個套件以進行最後的簡易測試網頁

In [ ]:
!pip install gradio

安裝完gradio後，我們需要import一些必須的套件，包括數據顯示和分析以及tensorflow提供的神經網路等等的套件

In [ ]:
%matplotlib inline

# 常用的數據分析和畫圖套件
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

# tensorflow其中的一些神經網路套件
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD

# 互動設計用的套件
from ipywidgets import interact_manual

# 剛剛安裝的Gradio
import gradio as gr

# 2.讀入資料集

老師這一次使用的是MNIST這個dataset，我們可以從keras中輕鬆的讀進來

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

雖然說MNIST這個dataset很有名且不複雜，其實並不用去分析裡面的data，但一般建立模型的時候先去了解資料的樣子是很重要的，因此這邊我還是先來稍微看一下裡面的資料的樣子

In [ ]:
print(f'資料集的Shape: {x_train.shape}')
print(f'資料的樣子: {x_train}')

資料集的Shape: (60000, 28, 28)
資料的樣子: [[[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 ...

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]]


可以看到MNIST的data是一個28x28的矩陣，共有六萬筆資料

# 3.事前處理

由於資料本身是28x28的二維矩陣，而我們要建立的標準神經網路只能應付一維的資料，因此我們需要先將所有資料reshape成28x28=784的樣子。而資料本身的數值是一個greyscale的pixel values，range是從0到255，因此我們把它除以255就可以將數值變為0到1之間的數值，我問GPT後得知了這樣做可以幫助模型訓練得更快和可以幫助模型的收斂。

In [ ]:
x_train = x_train.reshape(60000, 784)/255
x_test = x_test.reshape(10000, 784)/255

接著因為我們要處理的手寫辨識是一個分類問題，因此我們需要進行one-hot-encoding

In [ ]:
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# 4.開始建構神經網路

我們對資料做好事前的處理後，就可以開始建構我們的神經網路了。首先先打開一個標準空白的神經網路，接著利用add( )一步一步的建立我們神經網路的每一層。由於第一層的時候，TensorFlow不知道我們的參數長怎樣，因此需要我們告訴它，但之後的每一層的input就是前一層的output，因此不需要我們去告訴它了。

In [ ]:
N1 = 20
N2 = 20
N3 = 20
N4 = 20

model = Sequential()
model.add(Dense(N1, input_dim=784, activation='relu'))
model.add(Dense(N2, activation='relu'))
model.add(Dense(N3, activation='relu'))
model.add(Dense(N4, activation='relu'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


接著我們還差一個output層，我們的結果是0-9共十個，因此這個output層的神經元數便是10，同時使用softmax當激發函數以使所有結果的幾率加起來是1。

In [ ]:
model.add(Dense(10, activation='softmax'))

接下來就可以去實際建構我們的神經網路了，我們使用mse當做loss function，SGD為optimizer

In [ ]:
model.compile(loss='mse', optimizer=SGD(learning_rate=0.087), metrics=['accuracy'])

# 5.測試結果

結構好我們的網路後，可以開始訓練並看看我們的模型表現如何了

In [ ]:
model.fit(x_train, y_train, batch_size=100, epochs=10)

Epoch 1/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.0633 - loss: 0.0899
Epoch 2/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.1461 - loss: 0.0893
Epoch 3/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.2164 - loss: 0.0881
Epoch 4/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.2746 - loss: 0.0831
Epoch 5/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.4695 - loss: 0.0711
Epoch 6/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5661 - loss: 0.0584
Epoch 7/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.6775 - loss: 0.0462
Epoch 8/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7832 - loss: 0.0342
Epoch 9/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8249 - loss: 0.0267
Epoch 10/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8456 - loss: 0.0235


In [ ]:
score = model.evaluate(x_test, y_test)
print('loss:', score[0])
print('正確率', score[1])

313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8426 - loss: 0.0235
loss: 0.02059396170079708
正確率 0.8633000254631042


# 6.測試不同參數及function的結果如何

In [ ]:
N1 = 20
N2 = 20
N3 = 20
N4 = 20

model = Sequential()
model.add(Dense(N1, input_dim=784, activation='relu'))
model.add(Dense(N2, activation='relu'))
model.add(Dense(N3, activation='relu'))
model.add(Dense(N4, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=SGD(learning_rate=0.087), metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=100, epochs=10)

score = model.evaluate(x_test, y_test)
print('loss:', score[0])
print('正確率', score[1])

Epoch 1/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.5015 - loss: 1.4121
Epoch 2/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9022 - loss: 0.3302
Epoch 3/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9269 - loss: 0.2467
Epoch 4/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9401 - loss: 0.2006
Epoch 5/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9467 - loss: 0.1799
Epoch 6/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9516 - loss: 0.1654
Epoch 7/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9561 - loss: 0.1503
Epoch 8/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9587 - loss: 0.1390
Epoch 9/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9606 - loss: 0.1344
Epoch 10/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9622 - loss: 0.1267
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9535 - loss: 0.1564
loss: 0.13871487975120544
正確率 0.9603000283241272


可以看到換成categorical crossentropy這個loss function後，我們模型的表現變好了很多，因為CCE就是非常適合分類問題的，它可以最大化分類到正確答案的幾率

In [ ]:
from tensorflow.keras.optimizers import Adam
N1 = 20
N2 = 20
N3 = 20
N4 = 20

model = Sequential()
model.add(Dense(N1, input_dim=784, activation='relu'))
model.add(Dense(N2, activation='relu'))
model.add(Dense(N3, activation='relu'))
model.add(Dense(N4, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=100, epochs=10)

score = model.evaluate(x_test, y_test)
print('loss:', score[0])
print('正確率', score[1])

Epoch 1/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.6392 - loss: 1.1070
Epoch 2/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9220 - loss: 0.2695
Epoch 3/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9401 - loss: 0.2050
Epoch 4/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9487 - loss: 0.1714
Epoch 5/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9541 - loss: 0.1535
Epoch 6/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9563 - loss: 0.1450
Epoch 7/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9608 - loss: 0.1306
Epoch 8/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9620 - loss: 0.1237
Epoch 9/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9632 - loss: 0.1207
Epoch 10/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9682 - loss: 0.1053
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9536 - loss: 0.1537
loss: 0.14126631617546082
正確率 0.9570000171661377


可以看到將optimizer換成Adam後模型的表現沒有明顯變化。看來原本的learning rate就足夠了。

In [ ]:
N1 = 384
N2 = 384
N3 = 384
N4 = 384

model = Sequential()
model.add(Dense(N1, input_dim=784, activation='relu'))
model.add(Dense(N2, activation='relu'))
model.add(Dense(N3, activation='relu'))
model.add(Dense(N4, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=SGD(learning_rate=0.087), metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=100, epochs=10)

score = model.evaluate(x_test, y_test)
print('loss:', score[0])
print('正確率', score[1])

Epoch 1/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7701 - loss: 0.7672
Epoch 2/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9512 - loss: 0.1593
Epoch 3/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9677 - loss: 0.1025
Epoch 4/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9785 - loss: 0.0701
Epoch 5/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9838 - loss: 0.0543
Epoch 6/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9873 - loss: 0.0419
Epoch 7/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9902 - loss: 0.0336
Epoch 8/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9925 - loss: 0.0241
Epoch 9/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9951 - loss: 0.0180
Epoch 10/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9964 - loss: 0.0132
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9769 - loss: 0.0779
loss: 0.06765162199735641
正確率 0.9803000092506409


以下是一些調整神經元數量的嘗試得出的結果:
*   (64,64,64,64): 0.967
*   (128,64,32,16): 0.973
*   (256,128,64,32): 0.978
*   (128,128,128,128): 0.978
*   (256,256,256,256): 0.980
*   (384,384,384,384): 0.980

根據以上的測試，看起來256,256,256,256的神經元就足夠抓取圖像中的特徵了，更多的神經元有可能會造成overfitting的問題。


In [ ]:
N1 = 256
N2 = 256
N3 = 256
N4 = 256

model = Sequential()
model.add(Dense(N1, input_dim=784, activation='relu'))
model.add(Dense(N2, activation='relu'))
model.add(Dense(N3, activation='relu'))
model.add(Dense(N4, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=SGD(learning_rate=0.087), metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=20, epochs=10)

score = model.evaluate(x_test, y_test)
print('loss:', score[0])
print('正確率', score[1])

Epoch 1/10
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - accuracy: 0.8542 - loss: 0.4559
Epoch 2/10
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.9678 - loss: 0.1021
Epoch 3/10
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - accuracy: 0.9787 - loss: 0.0681
Epoch 4/10
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.9846 - loss: 0.0497
Epoch 5/10
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - accuracy: 0.9874 - loss: 0.0382
Epoch 6/10
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - accuracy: 0.9901 - loss: 0.0306
Epoch 7/10
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - accuracy: 0.9935 - loss: 0.0201
Epoch 8/10
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - accuracy: 0.9950 - loss: 0.0167
Epoch 9/10
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - accuracy: 0.9955 - loss: 0.0143
Epoch 10/10
3000/3000 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - accuracy: 0.9956 - loss: 0.0138
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9795 - loss: 0.0830
loss: 0.0747981145977974
正確率 0.981199979

以下是測試batch size跟epochs的影響:
*   (64,15): 0.977
*   (64,10): 0.979
*   (32,10): 0.982
*   (32,15): 0.983
*   (100,15): 0.977
*   (20,10): 0.981

可以看到batch size跟epochs的改變，讓模型的表現再進步了一點點，畢竟0.98已經很高了，有一點點的進步已經是很難的事情了



In [ ]:
from tensorflow.keras.layers import Dropout

N1 = 256
N2 = 256
N3 = 256
N4 = 256

model = Sequential()
model.add(Dense(N1, input_dim=784, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(N2, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(N3, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(N4, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=SGD(learning_rate=0.087), metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=32, epochs=15)

score = model.evaluate(x_test, y_test)
print('loss:', score[0])
print('正確率', score[1])

Epoch 1/15
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - accuracy: 0.7970 - loss: 0.6231
Epoch 2/15
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.9489 - loss: 0.1675
Epoch 3/15
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9655 - loss: 0.1166
Epoch 4/15
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9709 - loss: 0.0937
Epoch 5/15
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9751 - loss: 0.0814
Epoch 6/15
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9774 - loss: 0.0738
Epoch 7/15
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9777 - loss: 0.0691
Epoch 8/15
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9809 - loss: 0.0558
Epoch 9/15
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9828 - loss: 0.0557
Epoch 10/15
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9835 - loss: 0.0498
Epoch 11/15
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9866 - loss: 0.0435
Epoch 12/15
1875/1875 ━━━━━━━━

由於更多的神經元有可能導致overfitting的問題，因此我這邊加上Dropout去測試看看有沒有進步，可以看到結果並沒有明顯變化，甚至有一點退步。

# 7.Gradio測試

In [ ]:
N1 = 256
N2 = 256
N3 = 256
N4 = 256

model = Sequential()
model.add(Dense(N1, input_dim=784, activation='relu'))
model.add(Dense(N2, activation='relu'))
model.add(Dense(N3, activation='relu'))
model.add(Dense(N4, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=SGD(learning_rate=0.087), metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=32, epochs=15)

score = model.evaluate(x_test, y_test)
print('loss:', score[0])
print('正確率', score[1])

Epoch 1/15
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8313 - loss: 0.5313
Epoch 2/15
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9663 - loss: 0.1112
Epoch 3/15
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9780 - loss: 0.0696
Epoch 4/15
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9827 - loss: 0.0528
Epoch 5/15
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9895 - loss: 0.0349
Epoch 6/15
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9902 - loss: 0.0300
Epoch 7/15
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.9924 - loss: 0.0242
Epoch 8/15
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9941 - loss: 0.0173
Epoch 9/15
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9959 - loss: 0.0137
Epoch 10/15
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9963 - loss: 0.0120
Epoch 11/15
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9959 - loss: 0.0120
Epoch 12/15
1875/1875 ━━━━━━━━

In [ ]:
def resize_image(pic):
    image = np.array(pic["layers"][0], dtype=np.float32)
    image = image.astype(np.uint8)
    image_pil = Image.fromarray(image)
    background = Image.new("RGB", image_pil.size, (255, 255, 255))
    background.paste(image_pil, mask=image_pil.split()[3])
    image_pil = background
    image_gray = image_pil.convert("L")
    img_array = np.array(image_gray.resize((28, 28), resample=Image.LANCZOS))
    img_array = 255 - img_array
    img_array = img_array.reshape(1, 784) / 255.0
    return img_array

def recognize_digit(pic):
    img_array = resize_image(pic)
    prediction = model.predict(img_array).flatten()
    labels = list('0123456789')
    return {labels[i]: float(prediction[i]) for i in range(10)}

iface = gr.Interface(
    fn=recognize_digit,
    inputs=gr.Sketchpad(),
    outputs=gr.Label(num_top_classes=3),
    title="MNIST 手寫辨識",
    description="請在畫板上繪製數字"
)

iface.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://6a9a496e8d48d8a9ba.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://6a9a496e8d48d8a9ba.gradio.live
